<a href="https://colab.research.google.com/github/nick-nagaraj/face-countt/blob/master/Face_Nod_Shake_Nikhil_Nagaraj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install face-alignment

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import face_alignment
from skimage import io
import cv2
import numpy as np

#imports the face_detection model
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cuda')

#initializes head_shake_arrays to 0. Few values are added to prevent it from going out of range
head_shake_arr_no = [0,0,0]
head_shake_arr_yes = [0,0,0]

#Captures the video and sets up the config to write it to a seperate file called "output.avi". Also change the path of the video as required.
cap = cv2.VideoCapture('/gdrive/My Drive/AI_Work/Face_Video_1.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0,(int(cap.get(3)),int(cap.get(4))))

#Read the first frame in the video and derive center of video with respect to which all nods and shakes are detected.
ret, input = cap.read()

#My video happened to be rotated to the right causing the facial detector to fail. Remove the following two lines below this comment for your own video.
M = cv2.getRotationMatrix2D((input.shape[1]/2,input.shape[0]/2),90,1)
input = cv2.warpAffine(input,M,(input.shape[1],input.shape[0]))

#Get the feature points
preds = fa.get_landmarks(input)
preds = np.array(preds)

#Get the center feature point. AKA "nose"
i = preds[0][30]
cv2.circle(input, (int(i[0]),int(i[1])) ,3, (0,0,255), -1)
cv2.putText(input,"{}".format("1"),(int(i[0]),int(i[1])),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),2,cv2.LINE_AA)

#Assigns the center of the frame with the below values
sumX = int(i[0])
sumY = int(i[1])

#Draws lines for visual reference as which counts as a shake
cv2.line(input,(sumX-30,0),(sumX-30,input.shape[1]),(255,0,0),3)
cv2.line(input,(sumX+30,0),(sumX+30,input.shape[1]),(255,0,0),3)

#Read the rest of the frames
while True:
  head_shake_no = 0
  head_shake_yes = 0
  ret, input = cap.read()
  if ret==True:
      #Rotate to the left again, procedure remains same as above.
          M = cv2.getRotationMatrix2D((input.shape[1]/2,input.shape[0]/2),90,1)
          input = cv2.warpAffine(input,M,(input.shape[1],input.shape[0]))
          preds = fa.get_landmarks(input)
          preds = np.array(preds)
          i = preds[0][30]
          cv2.circle(input, (int(i[0]),int(i[1])) ,3, (0,0,255), -1)
          cv2.putText(input,"{}".format("1"),(int(i[0]),int(i[1])),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),2,cv2.LINE_AA)

          #Assigns the coordinates of the required feature to p0(x) and p1(y).
          p0 = int(i[0])
          p1 = int(i[1])

          #Criteria for nod and shake
          while ((p0 < sumX - 30 or p0 > sumX + 30) and (p1 > sumY - 40)):
            head_shake_no += 1
            break

          while (p1 > sumY + 40):
            head_shake_yes += 1
            break

          #Append head_shake values to head_shake array
          head_shake_arr_no.append(head_shake_no)
          head_shake_arr_yes.append(head_shake_yes)

          #Draw reference lines on each frame and rotate back.
          cv2.line(input,(sumX-30,0),(sumX-30,input.shape[1]),(255,0,0),3)
          cv2.line(input,(sumX+30,0),(sumX+30,input.shape[1]),(255,0,0),3)
          M = cv2.getRotationMatrix2D((input.shape[1]/2,input.shape[0]/2),270,1)
          input = cv2.warpAffine(input,M,(input.shape[1],input.shape[0]))
          #Write the image.
          out.write(input)

  else:
    break
counter_no = 0
counter_yes = 0

#Interprets the array to remove false detections, and count genuine nods/shakes
for i in range(len(head_shake_arr_no) - 1):
    if (head_shake_arr_no[i] == 0 and head_shake_arr_no[i+1] == 1 and head_shake_arr_no[i+2] == 1):
        counter_no += 0.25
    elif (head_shake_arr_no[i] == 1 and head_shake_arr_no[i+1] == 0 and head_shake_arr_no[i+2] == 0):
        counter_no += 0.25
for i in range(len(head_shake_arr_yes) - 1):
    if (head_shake_arr_yes[i] == 0 and head_shake_arr_yes[i+1] == 1 and head_shake_arr_yes[i+2] == 1):
        counter_yes += 1

print ("The number of shakes(no) is: {}".format(int(counter_no + 1)))
print ("The number of nods(yes) is: {}".format(counter_yes))
cap.release()
out.release()


The number of shakes(no) is: 2
The number of nods(yes) is: 4
